In [ ]:
%pip install nltk transformers torch annoy seaborn matplotlib scikit-learn PyPDF2 plotly


In [ ]:
import nltk
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertModel
import PyPDF2
import os
import pickle
from tqdm import tqdm
from huggingface_hub import snapshot_download
import os
import numpy as np
import PyPDF2
from annoy import AnnoyIndex



from collections import defaultdict
from tqdm import tqdm
from multiprocessing import Pool
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from torch.nn import DataParallel
import torch
from paper_processing_for_embeddings import preprocess_and_read 

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
# Check if CUDA is available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

# If multiple GPUs are available, use DataParallel
if torch.cuda.device_count() > 1:
    model = DataParallel(model)

model = model.to(device)



In [ ]:
def embed_words_batch(words):
    inputs = tokenizer(words, padding=True, return_tensors='pt', truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move inputs to the appropriate device
    outputs = model(**inputs)
    return outputs.hidden_states[-1][:, 0, :].detach().cpu().numpy()


In [ ]:
def build_annoy_index(embeddings_dict):
    f = list(embeddings_dict.values())[0]['embedding'].shape[0]
    t = AnnoyIndex(f, 'angular')
    for i, (word, data) in enumerate(embeddings_dict.items()):
        t.add_item(i, data['embedding'])
    t.build(10)
    return t

def query_similar_words(query, index, embeddings_dict, top_n=5):
    query_embedding = embed_words_batch([query])[0]  # Embed the query word
    nearest_ids = index.get_nns_by_vector(query_embedding, top_n)

    similar_words_with_titles = []
    for i in nearest_ids:
        word = list(embeddings_dict.keys())[i]
        title = embeddings_dict[word]['file'].split('/')[-1]  # Extract the file name
        similar_words_with_titles.append((word, title))

    return similar_words_with_titles



In [ ]:
with open('word_embeddings.pkl', 'rb') as f:
    embeddings_dict = pickle.load(f)


In [ ]:
# Build Annoy index
annoy_index = build_annoy_index(embeddings_dict)

In [ ]:
def normalize_text(text):
    """
    Enhances normalization of text by lowercasing, replacing hyphens and underscores, 
    and removing non-alphanumeric characters.
    """
    text = text.lower()
    text = re.sub(r'[\-\_]', ' ', text)  # Replace hyphens and underscores with spaces
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Remove non-alphanumeric characters
    text = ' '.join(text.split())  # Remove extra spaces
    return text

def user_approval(technique):
    """
    Asks for user approval for a new technique.
    """
    response = input(f"Approve technique '{technique}'? (y/n): ").strip().lower()
    return response == 'y'


def find_nearest_neighbors(techniques, embeddings_dict, annoy_index, depth=0, max_depth=10):
    """
    Recursively finds nearest neighbors for a list of techniques up to a certain depth.
    Each new technique requires user approval.
    """
    if depth >= max_depth:
        print(f"Maximum depth reached: {max_depth}")
        return techniques

    new_techniques = set()
    for technique in techniques:
        similar_words = query_similar_words(technique, annoy_index, embeddings_dict, top_n=10)
        for word, _ in similar_words:
            normalized_word = normalize_text(word)
            if normalized_word not in techniques:
                # Present the normalized word for user approval
                if user_approval(normalized_word):
                    new_techniques.add(normalized_word)

    # Check for new techniques
    if not new_techniques:
        print("No new approved techniques found")
        return techniques

    # Add new techniques and recurse
    updated_techniques = techniques.union(new_techniques)
    print(f"Depth {depth}: Found {len(new_techniques)} new approved techniques")
    return find_nearest_neighbors(updated_techniques, embeddings_dict, annoy_index, depth+1, max_depth)

# Example usage
initial_techniques = {"chain-of-thought", "few-shot"}  # Replace with your initial set of techniques
max_depth = 5  # You can adjust this to control the recursion depth
all_techniques = find_nearest_neighbors(initial_techniques, embeddings_dict, annoy_index, max_depth=max_depth)

print("All approved techniques found:")
for technique in all_techniques:
    print(technique)